In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import json
from pprint import pprint
import requests

from sqlalchemy import create_engine

url = 'http://opendata-ajuntament.barcelona.cat/data/api/action/datastore_search?resource_id=2d190658-93ac-4c43-a23f-c5d313b1ae9c&limit=5'
                                            
bsObj = BeautifulSoup(urlopen(url))


In [2]:
# Source 1: Store CSV into DataFrame
csv_file = "./Resources/2019_01_Gener_qualitat_aire_BCN.csv"
aire_data_df = pd.read_csv(csv_file)
aire_data_df.head()

,nom_cabina,qualitat_aire,codi_dtes,zqa,codi_eoi,longitud,latitud,hora_o3,qualitat_o3,valor_o3,hora_no2,qualitat_no2,valor_no2,hora_pm10,qualitat_pm10,valor_pm10,generat,dateTime
0,Barcelona - Sants,Bona,ID,1,8019042,2.1331,41.3788,NaN,NaN,NaN,23h,Bona,86 µg/m³,NaN,NaN,NaN,01/01/2019 0:00,1546297502
1,Barcelona - Eixample,Regular,IH,1,8019043,2.1538,41.3853,23h,Bona,3 µg/m³,23h,Regular,112 µg/m³,23h,Bona,36 µg/m³,01/01/2019 0:00,1546297502
2,Barcelona - Gràcia,Regular,IJ,1,8019044,2.1534,41.3987,23h,Bona,3 µg/m³,23h,Regular,113 µg/m³,23h,Regular,39 µg/m³,01/01/2019 0:00,1546297502
3,Barcelona - Ciutadella,Bona,IL,1,8019050,2.1874,41.3864,23h,Bona,2 µg/m³,23h,Bona,74 µg/m³,NaN,NaN,NaN,01/01/2019 0:00,1546297502
4,Barcelona - Vall Hebron,Bona,IN,1,8019054,2.1480,41.4261,23h,Bona,10 µg/m³,23h,Bona,67 µg/m³,23h,Bona,12 µg/m³,01/01/2019 0:00,1546297502


In [3]:
# Source 2: 
# Extract Public transports of the city of Barcelona

import json
import pandas as pd
from pandas.io.json import json_normalize

url = 'http://opendata-ajuntament.barcelona.cat/data/api/action/datastore_search?resource_id=e07dec0d-4aeb-40f3-b987-e1f35e088ce2'

lm_json = requests.get(url).json()
lm_json.keys()
lm_json['result']['records']

df=json_normalize(lm_json['result']['records'])
list(df)

['ADRECA',
 'BARRI',
 'CAPA_GENERICA',
 'CODI_CAPA',
 'DISTRICTE',
 'ED50_COORD_X',
 'ED50_COORD_Y',
 'EQUIPAMENT',
 'ETRS89_COORD_X',
 'ETRS89_COORD_Y',
 'LATITUD',
 'LONGITUD',
 'NOM_BARRI',
 'NOM_CAPA',
 'NOM_DISTRICTE',
 'TELEFON',
 '_id']

In [4]:
df.head()

,ADRECA,BARRI,CAPA_GENERICA,CODI_CAPA,DISTRICTE,ED50_COORD_X,ED50_COORD_Y,EQUIPAMENT,ETRS89_COORD_X,ETRS89_COORD_Y,LATITUD,LONGITUD,NOM_BARRI,NOM_CAPA,NOM_DISTRICTE,TELEFON,_id
0,,23,Transports i serveis relacionats,K001,5,426477.889,4583649.008,FGC (L6) - REINA ELISENDA (Sortida Duquesa d'O...,426385.994,4583448.062,41.399203,2.11937,Sarrià,Metro i línies urbanes FGC,Sarrià-Sant Gervasi,,1
1,,26,Transports i serveis relacionats,K001,5,427818.595,4583478.761,FGC (L6) - LA BONANOVA-,427726.688,4583277.816,41.397791,2.135427,Sant Gervasi - Galvany,Metro i línies urbanes FGC,Sarrià-Sant Gervasi,,2
2,,53,Transports i serveis relacionats,K001,8,432051.444,4589400.176,METRO (L11) - CASA DE L'AIGUA (C. Vila-Real)-,431959.5,4589199.175,41.451492,2.185391,la Trinitat Nova,Metro i línies urbanes FGC,Nou Barris,,3
3,,55,Transports i serveis relacionats,K001,8,431149.447,4590452.022,METRO (L11) - CIUTAT MERIDIANA (C. Pedraforca)-,431057.511,4590251.011,41.460889,2.174473,Ciutat Meridiana,Metro i línies urbanes FGC,Nou Barris,,4
4,,7,Transports i serveis relacionats,K001,2,430579.417,4582275.812,METRO (L1) - CATALUNYA (Rda. Universitat)-,430487.484,4582074.877,41.3872,2.168588,la Dreta de l'Eixample,Metro i línies urbanes FGC,Eixample,,5


In [5]:
# Data clean

transports_data_df = df[[  'ADRECA'         \
                         , 'CAPA_GENERICA'  \
                         , 'CODI_CAPA'      \
                         , 'NOM_CAPA'       \
                         , 'LONGITUD'       \
                         , 'LATITUD'        \
                         , 'EQUIPAMENT'     \
                         , 'DISTRICTE'      \
                         , 'NOM_BARRI']].copy()



transports_data_df.columns=['ID', 'tipo', 'codigo_sub_tipo','sub_tipo' \
                            , 'longitud','latitud', 'equipo','distrito','barrio']

transports_data_df

,ID,tipo,codigo_sub_tipo,sub_tipo,longitud,latitud,equipo,distrito,barrio
0,,Transports i serveis relacionats,K001,Metro i línies urbanes FGC,2.11937,41.399203,FGC (L6) - REINA ELISENDA (Sortida Duquesa d'O...,5,Sarrià
1,,Transports i serveis relacionats,K001,Metro i línies urbanes FGC,2.135427,41.397791,FGC (L6) - LA BONANOVA-,5,Sant Gervasi - Galvany
2,,Transports i serveis relacionats,K001,Metro i línies urbanes FGC,2.185391,41.451492,METRO (L11) - CASA DE L'AIGUA (C. Vila-Real)-,8,la Trinitat Nova
3,,Transports i serveis relacionats,K001,Metro i línies urbanes FGC,2.174473,41.460889,METRO (L11) - CIUTAT MERIDIANA (C. Pedraforca)-,8,Ciutat Meridiana
4,,Transports i serveis relacionats,K001,Metro i línies urbanes FGC,2.168588,41.3872,METRO (L1) - CATALUNYA (Rda. Universitat)-,2,la Dreta de l'Eixample
5,,Transports i serveis relacionats,K001,Metro i línies urbanes FGC,2.168507,41.387706,METRO (L1) - CATALUNYA (Rambla de Catalunya)-,2,la Dreta de l'Eixample
6,,Transports i serveis relacionats,K001,Metro i línies urbanes FGC,2.118998,41.398967,FGC (L6) - REINA ELISENDA-,5,Sarrià
7,,Transports i serveis relacionats,K001,Metro i línies urbanes FGC,2.083994,41.362753,METRO (L5) - SANT ILDEFONS-,None,
8,,Transports i serveis relacionats,K001,Metro i línies urbanes FGC,2.142987,41.424923,"METRO (L3, L5) - VALL D'HEBRON (C. de les Bass...",7,la Vall d'Hebron
9,,Transports i serveis relacionats,K002,Ferrocarrils Generalitat (FGC),2.158326,41.392331,FGC - PROVENÇA (C. de Provença)-,2,l'Antiga Esquerra de l'Eixample


In [6]:
# Extract: Air quality measure stations of the city of Barcelona

url = 'http://opendata-ajuntament.barcelona.cat/data/api/action/datastore_search?resource_id=6c33f60b-2f52-4954-a5e6-cf879890551c'

lm_json = requests.get(url).json()
lm_json.keys()
lm_json['result']['records']

df=json_normalize(lm_json['result']['records'])
df

,Codi_Barri,Codi_Districte,Contaminant_1,Contaminant_2,Contaminant_3,Emissions_Properes,Nom_Barri,Nom_Districte,Ocupacio_sol,Ubicacio,_id,codi_dtes,codi_eoi,latitud,longitud,nom_cabina,zqa
0,4,1,NO2,O3,,Fons,"Sant Pere, Santa Caterina i la Ribera",Ciutat Vella,Urbana,Parc de la Ciutadella,1,IL,8019050,41.3864,2.1874,Barcelona - Ciutadella,1
1,9,5,NO2,O3,PM10,Trànsit,la Nova Esquerra de l'Eixample,Eixample,Urbana,Av. Roma - c/ Comte Urgell,2,IH,8019043,41.3853,2.1538,Barcelona - Eixample,1
2,31,6,NO2,O3,PM10,Trànsit,la Vila de Gracia,Gracia,Urbana,Plaça Gal·la Placídia (Via Augusta - Travesser...,3,IJ,8019044,41.3987,2.1534,Barcelona - Gràcia,1
3,21,4,NO2,O3,PM10,Fons,Pedralbes,Les Corts,Urbana,c/ John Maynard Keynes - c/ de Jordi Girona,4,IZ,8019057,41.3875,2.1151,Barcelona - Palau Reial,1
4,68,10,NO2,,PM10,Fons,el Poblenou,Sant Marti,Urbana,Plaça Josep Trueta (Pujades - Lope de Vega),5,I2,8019004,41.4039,2.2045,Barcelona - Poblenou,1
5,18,3,NO2,,,Fons,Sants,Sants-Montjuic,Urbana,Jardins de Can Mantega (Joan Güell - Violant d...,6,ID,8019042,41.3788,2.1331,Barcelona - Sants,1
6,41,7,NO2,O3,PM10,Fons,la Vall d'Hebron,Horta-Guinardo,Urbana,Parc de la Vall d’Hebron - c/ Martí Codolar - ...,7,IN,8019054,41.4261,2.1480,Barcelona - Vall Hebron,1


In [7]:
# Data clean

stations_data_df = df[[  '_id'              \
                         , 'nom_cabina'     \
                         , 'longitud'       \
                         , 'latitud'        \
                         , 'Ubicacio'       \
                         , 'Nom_Districte'  \
                         , 'Nom_Barri']].copy()

stations_data_df.columns=['ID', 'estacion', 'longitud','latitud','ubicacion' \
                          , 'distrito','barrio']
stations_data_df.head()

,ID,estacion,longitud,latitud,ubicacion,distrito,barrio
0,1,Barcelona - Ciutadella,2.1874,41.3864,Parc de la Ciutadella,Ciutat Vella,"Sant Pere, Santa Caterina i la Ribera"
1,2,Barcelona - Eixample,2.1538,41.3853,Av. Roma - c/ Comte Urgell,Eixample,la Nova Esquerra de l'Eixample
2,3,Barcelona - Gràcia,2.1534,41.3987,Plaça Gal·la Placídia (Via Augusta - Travesser...,Gracia,la Vila de Gracia
3,4,Barcelona - Palau Reial,2.1151,41.3875,c/ John Maynard Keynes - c/ de Jordi Girona,Les Corts,Pedralbes
4,5,Barcelona - Poblenou,2.2045,41.4039,Plaça Josep Trueta (Pujades - Lope de Vega),Sant Marti,el Poblenou


In [8]:
# Bus stops of the city of Barcelona

url = 'http://opendata-ajuntament.barcelona.cat/data/api/action/datastore_search?resource_id=2d190658-93ac-4c43-a23f-c5d313b1ae9c'

lm_json = requests.get(url).json()
lm_json.keys()
lm_json['result']['records']

df=json_normalize(lm_json['result']['records'])
df

,ADRECA,BARRI,CAPA_GENERICA,CODI_CAPA,DISTRICTE,ED50_COORD_X,ED50_COORD_Y,EQUIPAMENT,ETRS89_COORD_X,ETRS89_COORD_Y,LATITUD,LONGITUD,NOM_BARRI,NOM_CAPA,NOM_DISTRICTE,TELEFON,_id
0,,35,Transports i serveis relacionats,K014,7,430860.98,4585220.256,BUS -192--,430769.045,4585019.294,41.413744,2.171619,el Guinardó,Autobusos diürns,Horta-Guinardó,,1
1,,28,Transports i serveis relacionats,K014,6,427799.56,4585969.413,BUS -124--,427707.654,4585768.445,41.420222,2.134902,Vallcarca i els Penitents,Autobusos diürns,Gràcia,,2
2,,36,Transports i serveis relacionats,K014,7,430143.537,4586275.609,BUS -117--,430051.609,4586074.637,41.423187,2.162913,la Font d'en Fargues,Autobusos diürns,Horta-Guinardó,,3
3,,36,Transports i serveis relacionats,K014,7,430206.255,4586243.056,BUS -117--,430114.327,4586042.085,41.422899,2.163667,la Font d'en Fargues,Autobusos diürns,Horta-Guinardó,,4
4,,23,Transports i serveis relacionats,K014,5,426545.998,4583426.929,BUS -130--,426454.103,4583225.985,41.397209,2.120212,Sarrià,Autobusos diürns,Sarrià-Sant Gervasi,,5
5,,27,Transports i serveis relacionats,K014,5,428127.012,4584951.047,BUS -131--,428035.102,4584750.088,41.411079,2.138941,el Putxet i el Farró,Autobusos diürns,Sarrià-Sant Gervasi,,6
6,,27,Transports i serveis relacionats,K014,5,428298.16,4584996.619,BUS -131--,428206.249,4584795.66,41.411505,2.140984,el Putxet i el Farró,Autobusos diürns,Sarrià-Sant Gervasi,,7
7,,25,Transports i serveis relacionats,K014,5,427912.005,4584982.5,BUS -196--,427820.097,4584781.541,41.411343,2.136365,Sant Gervasi - la Bonanova,Autobusos diürns,Sarrià-Sant Gervasi,,8
8,,69,Transports i serveis relacionats,K014,10,434045.819,4584565.646,BUS -V29--,433953.855,4584364.689,41.408116,2.209794,Diagonal Mar i el Front Marítim del Poblenou,Autobusos diürns,Sant Martí,,9
9,,16,Transports i serveis relacionats,K014,3,427580.37,4580199.715,BUS -115--,427488.464,4579998.8,41.368237,2.13297,la Bordeta,Autobusos diürns,Sants-Montjuïc,,10


In [9]:
# Data clean

bus_stops_data_df = df[[  'ADRECA'         \
                         , 'CAPA_GENERICA'  \
                         , 'CODI_CAPA'      \
                         , 'NOM_CAPA'       \
                         , 'LONGITUD'       \
                         , 'LATITUD'        \
                         , 'EQUIPAMENT'     \
                         , 'DISTRICTE'      \
                         , 'NOM_BARRI']].copy()
bus_stops_data_df.columns=['ID', 'tipo', 'codigo_sub_tipo','sub_tipo' \
                            , 'longitud','latitud', 'equipo','distrito','barrio']
bus_stops_data_df.head()

,ID,tipo,codigo_sub_tipo,sub_tipo,longitud,latitud,equipo,distrito,barrio
0,,Transports i serveis relacionats,K014,Autobusos diürns,2.171619,41.413744,BUS -192--,7,el Guinardó
1,,Transports i serveis relacionats,K014,Autobusos diürns,2.134902,41.420222,BUS -124--,6,Vallcarca i els Penitents
2,,Transports i serveis relacionats,K014,Autobusos diürns,2.162913,41.423187,BUS -117--,7,la Font d'en Fargues
3,,Transports i serveis relacionats,K014,Autobusos diürns,2.163667,41.422899,BUS -117--,7,la Font d'en Fargues
4,,Transports i serveis relacionats,K014,Autobusos diürns,2.120212,41.397209,BUS -130--,5,Sarrià


In [10]:
# Database connectio
rds_connection_string = "golivera:golivera@127.0.0.1/animals?charset=utf8"
engine = create_engine(f'mysql://{rds_connection_string}', encoding = 'utf-8')

In [11]:
engine.table_names()

['bus_stops',
 'calidad_aire',
 'cookies',
 'customer_location',
 'customer_name',
 'globalfirepower',
 'line_items',
 'movies',
 'music',
 'orders',
 'people',
 'players',
 'seasons_stats',
 'stations',
 'stories',
 'tf_group',
 'tf_permission',
 'tf_user',
 'tf_user_group',
 'transports',
 'users',
 'wordassociation',
 'wordassociation_ac']

In [12]:
# Load transports data

transports_data_df.to_sql(name='transports', con=engine, if_exists='append', index=False)

In [13]:
# Load bus stops data

bus_stops_data_df.to_sql(name='bus_stops', con=engine, if_exists='append', index=False)

In [14]:
# Load stations data

stations_data_df.to_sql(name='stations', con=engine, if_exists='append', index=False)

In [15]:
# Load air quality data

aire_data_df.to_sql(name='calidad_aire', con=engine, if_exists='append', index=False)

In [16]:
# Query transports data

pd.read_sql_query('select * from transports', con=engine).head()

,ID,tipo,codigo_sub_tipo,sub_tipo,longitud,latitud,equipo,distrito,barrio
0,,Transports i serveis relacionats,K001,Metro i línies urbanes FGC,2.11937,41.399203,FGC (L6) - REINA ELISENDA (Sortida Duquesa d'O...,5,Sarrià
1,,Transports i serveis relacionats,K001,Metro i línies urbanes FGC,2.135427,41.397791,FGC (L6) - LA BONANOVA-,5,Sant Gervasi - Galvany
2,,Transports i serveis relacionats,K001,Metro i línies urbanes FGC,2.185391,41.451492,METRO (L11) - CASA DE L'AIGUA (C. Vila-Real)-,8,la Trinitat Nova
3,,Transports i serveis relacionats,K001,Metro i línies urbanes FGC,2.174473,41.460889,METRO (L11) - CIUTAT MERIDIANA (C. Pedraforca)-,8,Ciutat Meridiana
4,,Transports i serveis relacionats,K001,Metro i línies urbanes FGC,2.168588,41.3872,METRO (L1) - CATALUNYA (Rda. Universitat)-,2,la Dreta de l'Eixample


In [17]:
# Query bus stops data

pd.read_sql_query('select * from bus_stops', con=engine).head()

,ID,tipo,codigo_sub_tipo,sub_tipo,longitud,latitud,equipo,distrito,barrio
0,,Transports i serveis relacionats,K014,Autobusos diürns,2.171619,41.413744,BUS -192--,7,el Guinardó
1,,Transports i serveis relacionats,K014,Autobusos diürns,2.134902,41.420222,BUS -124--,6,Vallcarca i els Penitents
2,,Transports i serveis relacionats,K014,Autobusos diürns,2.162913,41.423187,BUS -117--,7,la Font d'en Fargues
3,,Transports i serveis relacionats,K014,Autobusos diürns,2.163667,41.422899,BUS -117--,7,la Font d'en Fargues
4,,Transports i serveis relacionats,K014,Autobusos diürns,2.120212,41.397209,BUS -130--,5,Sarrià


In [18]:
# Query stations data

pd.read_sql_query('select * from stations', con=engine).head()

,ID,estacion,longitud,latitud,ubicacion,distrito,barrio
0,1,Barcelona - Ciutadella,2.1874,41.3864,Parc de la Ciutadella,Ciutat Vella,"Sant Pere, Santa Caterina i la Ribera"
1,2,Barcelona - Eixample,2.1538,41.3853,Av. Roma - c/ Comte Urgell,Eixample,la Nova Esquerra de l'Eixample
2,3,Barcelona - Gràcia,2.1534,41.3987,Plaça Gal·la Placídia (Via Augusta - Travesser...,Gracia,la Vila de Gracia
3,4,Barcelona - Palau Reial,2.1151,41.3875,c/ John Maynard Keynes - c/ de Jordi Girona,Les Corts,Pedralbes
4,5,Barcelona - Poblenou,2.2045,41.4039,Plaça Josep Trueta (Pujades - Lope de Vega),Sant Marti,el Poblenou


In [19]:
# Query air quality data

pd.read_sql_query('select * from calidad_aire', con=engine).head()

,nom_cabina,qualitat_aire,codi_dtes,zqa,codi_eoi,longitud,latitud,hora_o3,qualitat_o3,valor_o3,hora_no2,qualitat_no2,valor_no2,hora_pm10,qualitat_pm10,valor_pm10,generat,dateTime
0,Barcelona - Sants,Bona,ID,1,8019042,2.1331,41.3788,None,None,None,23h,Bona,86 µg/m³,None,None,None,01/01/2019 0:00,1546297502
1,Barcelona - Eixample,Regular,IH,1,8019043,2.1538,41.3853,23h,Bona,3 µg/m³,23h,Regular,112 µg/m³,23h,Bona,36 µg/m³,01/01/2019 0:00,1546297502
2,Barcelona - Gràcia,Regular,IJ,1,8019044,2.1534,41.3987,23h,Bona,3 µg/m³,23h,Regular,113 µg/m³,23h,Regular,39 µg/m³,01/01/2019 0:00,1546297502
3,Barcelona - Ciutadella,Bona,IL,1,8019050,2.1874,41.3864,23h,Bona,2 µg/m³,23h,Bona,74 µg/m³,None,None,None,01/01/2019 0:00,1546297502
4,Barcelona - Vall Hebron,Bona,IN,1,8019054,2.1480,41.4261,23h,Bona,10 µg/m³,23h,Bona,67 µg/m³,23h,Bona,12 µg/m³,01/01/2019 0:00,1546297502
